In [2]:
%load_ext autoreload

In [5]:
%autoreload 2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import uproot4
import pathlib
import seaborn as sns
import sys
import json
import seaborn as sns

# Jupyter Lab imports.
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, fixed

# Path to local imports.
sys.path.append("/home/drew/He6CRES/rocks_analysis_pipeline/")
from results import ExperimentResults
from rocks_utility import he6cres_db_query

# Path to local imports.
sys.path.append("/home/drew/He6CRES/he6-cres-spec-sims/")

# Local imports.
import he6_cres_spec_sims.spec_tools.spec_calc.spec_calc as sc
import he6_cres_spec_sims.experiment as exp
from he6_cres_spec_sims.experiment import ExpResults

# Spec Sims on Local Machine Demo. 

Author: Drew Byron 

Date: 11/17/22

Use this alongside the [readme.md](https://github.com/Helium6CRES/he6-cres-spec-sims/tree/feature/rocks_dev) in the he6-cres-spec-sims repo in order to run simulations of your own. 

## Step 0: Define the simulated experiment.

Grab the necessary config files from the examples in the repo (`he6-cres-spec-sims/config_files/`). Will need to change paths. Note that you can change these as you see fit but it is the name of the `.json` that specifies the name of the experiment directory so make that descriptive. 

Something like the following can be used to copy the example config files to a local directory suitable for containing the simulation results: 

In [4]:
!cp /home/drew/He6CRES/he6-cres-spec-sims/config_files/local* /media/drew/T7\ Shield/spec_sims_results/local_experiments

## Step 1: Run the simulated experiment.

Follow the instructions in the [readme.md](https://github.com/Helium6CRES/he6-cres-spec-sims/tree/feature/rocks_dev) for running the experiment.

The below is the function call I use (from within `he6-cres-spec-sims`). Change the paths and you're good to go.

`./run_local_experiment.py -exp "/media/drew/T7 Shield/spec_sims_results/local_experiments/local_exp_config_example.json"
`


## Step 2: Load the results. 

Now use the class `ExpResults` from the `experiment.py` module to grab the experiment results. 

There is a `.yaml` summary file that is created by the simulation run that is in the simulation directory that is built. There are now three different config files and I know that it's a bit confusing. But you can load the results of the experiment by pointing to that `.yaml` summary file which will have the form: `{experiment name}_exp.yaml` and will be inside the experiment directory.  

In [7]:
experiment_config_path = "/media/drew/T7 Shield/spec_sims_results/local_experiments/local_exp_config_example/local_exp_config_example_exp.yaml"
sim = ExpResults.load(experiment_config_path =experiment_config_path  )

/media/drew/T7 Shield/spec_sims_results/local_experiments/local_exp_config_example
+++++++++++++++++++++++++++++++++++++++++++++++++


Loading simulation 0 / 11


+++++++++++++++++++++++++++++++++++++++++++++++++
Optimization terminated successfully.
         Current function value: -3.253639
         Iterations: 43
         Function evaluations: 96
Trap width: (-0.04319391632080079,0.04319391632080079)
Maximum Field: 3.2536386230916876

Set field: 3.25, Trap current: 1.8

+++++++++++++++++++++++++++++++++++++++++++++++++


Loading simulation 1 / 11


+++++++++++++++++++++++++++++++++++++++++++++++++
Optimization terminated successfully.
         Current function value: -3.003359
         Iterations: 43
         Function evaluations: 95
Trap width: (-0.04319392015039922,0.04319392015039922)
Maximum Field: 3.0033587280747316

Set field: 3.0, Trap current: 1.661538

+++++++++++++++++++++++++++++++++++++++++++++++++


Loading simulation 2 / 11


+++++++++++++++++++++++++++++++++++++++++++

## Step 3: Tools for exploratory analysis. 

In [9]:
sim.__dict__

{'experiment_params': {'experiment_name': 'local_exp_config_example',
  'base_config_path': '/media/drew/T7 Shield/spec_sims_results/local_experiments/local_base_config_example.yaml',
  'isotope': 'He6',
  'events_to_simulate': -1,
  'betas_to_simulate': 10.0,
  'rand_seeds': [4062,
   3759,
   3456,
   3153,
   2850,
   2547,
   2244,
   1941,
   1638,
   1335,
   1032],
  'fields_T': [3.25, 3.0, 2.75, 2.5, 2.25, 2.0, 1.75, 1.5, 1.25, 1.0, 0.75],
  'traps_A': [1.8,
   1.661538,
   1.523077,
   1.384615,
   1.246154,
   1.107692,
   0.969231,
   0.830769,
   0.692308,
   0.553846,
   0.4153845]},
 'config_paths': [PosixPath('/media/drew/T7 Shield/spec_sims_results/local_experiments/local_exp_config_example/0_field_3.25T.yaml'),
  PosixPath('/media/drew/T7 Shield/spec_sims_results/local_experiments/local_exp_config_example/1_field_3.0T.yaml'),
  PosixPath('/media/drew/T7 Shield/spec_sims_results/local_experiments/local_exp_config_example/2_field_2.75T.yaml'),
  PosixPath('/media/drew/T7

In [10]:
sim.tracks.head()

energy     gamma   energy_stop  initial_rho_pos  initial_phi_pos  \
0  1.952178e+06  4.820317  1.659358e+06         0.002729       341.385374   
1  1.952178e+06  4.820317  1.659358e+06         0.002729       341.385374   
2  1.952178e+06  4.820317  1.659358e+06         0.002729       341.385374   
0  1.754479e+06  4.433429  1.616955e+06         0.003658       225.707633   
1  1.754479e+06  4.433429  1.616955e+06         0.003658       225.707633   

   initial_zpos  initial_theta  cos_initial_theta  initial_phi_dir  \
0     -0.001010      89.357093           0.011221       109.759904   
1     -0.001010      89.357093           0.011221       109.759904   
2     -0.001010      89.357093           0.011221       109.759904   
0      0.001582      89.246612           0.013149        98.712154   
1      0.001582      89.246612           0.013149        98.712154   

   center_theta  ...  energy_accept_high  energy_accept_low  \
0     89.316445  ...        2.071692e+06       1.922950e+06   
1     89.316445  ...        2.071692e+06       1.922950e+06   
2     89.316445  ...        2.071692e+06       1.922950e+06   
0     89.163092  ...        1.873023e+06       1.735724e+06   
1     89.163092  ...        1.873023e+06       1.735724e+06   

   gamma_accept_high  gamma_accept_low  time_start  time_stop    freq_start  \
0           5.054200          4.763120    0.000010   0.012539  7.458280e+08   
1           5.054200          4.763120    0.000010   0.012539  9.318048e+08   
2           5.054200          4.763120    0.000010   0.012539  1.117782e+09   
0           4.665415          4.396726    0.000019   0.008250  8.175208e+08   
1           4.665415          4.396726    0.000019   0.008250  1.000778e+09   

   simulation_num  field  trap_current  
0               0   3.25      1.800000  
1               0   3.25      1.800000  
2               0   3.25      1.800000  
0               1   3.00      1.661538  
1               1   3.00      1.661538  

[5 rows x 44 columns]

In [11]:
sim.tracks.columns

Index(['energy', 'gamma', 'energy_stop', 'initial_rho_pos', 'initial_phi_pos',
       'initial_zpos', 'initial_theta', 'cos_initial_theta', 'initial_phi_dir',
       'center_theta', 'cos_center_theta', 'initial_field', 'initial_radius',
       'center_x', 'center_y', 'rho_center', 'trapped_initial_theta',
       'max_radius', 'min_radius', 'avg_cycl_freq', 'b_avg', 'freq_stop',
       'zmax', 'axial_freq', 'mod_index', 'segment_power', 'slope',
       'segment_length', 'band_power', 'band_num', 'segment_num', 'event_num',
       'beta_num', 'fraction_of_spectrum', 'energy_accept_high',
       'energy_accept_low', 'gamma_accept_high', 'gamma_accept_low',
       'time_start', 'time_stop', 'freq_start', 'simulation_num', 'field',
       'trap_current'],
      dtype='object')

In [13]:
%matplotlib widget

df = sim.tracks
df_categorical = df.loc[:, df.apply(lambda x: x.nunique()) <= 20]
fields = sorted(df['field'].unique().tolist())

plt.rcParams['figure.dpi']= 100
@interact
def scatter_plots(
    field = widgets.Select(options = fields,  description='set_field: '),
    column_1 = widgets.Select(options = df.columns,value = 'energy', description='x_col: '),
    column_2 = widgets.Select(options = df.columns,value = 'b_avg', description='y_col: '),
    column_3 = widgets.Select(options = df_categorical.columns,value = 'field', description='hue_col: '),
    mrk_sz = widgets.FloatSlider(value=.4, min=0,max=1.0,step=1e-2),
    alpha = widgets.FloatSlider(value=.5, min=0.0,max=1.0,step=1e-2), 
    frac_pts = widgets.FloatSlider(value=0.02,min=0.0,max=1.0,step=1e-2), 
    bins = widgets.IntSlider(value=200,min=50,max=700,step=10),
    fix_field = widgets.Checkbox(
    value=False,
    description='fix field',
    disabled=False,
    indent=False
)
):
    
    plt.close("all")

    sns.lmplot(data=df, x=column_1, y=column_2, hue=column_3, fit_reg=False, scatter_kws={'s': mrk_sz, 'alpha': alpha})
    plt.show()

interactive(children=(Select(description='set_field: ', options=(0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2.25, 2.5, 2…